# Pipeline Hola Cetáceos
---
### Giovanny Alejandro Cuervo Londoño

En este notebook se pretende realizar el entubamiento del conjunto de datos de audios de la <a href="https://cis.whoi.edu/science/B/whalesounds/index.cfm\"> Watkins Marine Mammal Sound Database 


In [1]:
import numpy as np
import pandas as pd
import os
import librosa
import re
import tensorflow as tf

In [2]:
!python --version
print('NumPy', np.__version__)
print('Pandas', pd.__version__)
print('librosa', librosa.__version__)
print('re', re.__version__)
print('tensorflow', tf.__version__)

Python 3.7.10
NumPy 1.19.5
Pandas 1.1.5
librosa 0.8.0
re 2.2.1
tensorflow 2.4.1


Este notebook se creó con estas versiones de módulos:
> Python 3.7.10

> NumPy 1.19.5

> Pandas 1.1.5

> librosa 0.8.0

> re 2.2.1

> tensorflow 2.4.1

Conectamos nuestro directorio a de audios

In [3]:
#si esta trabajando en google drive
from google.colab import drive
drive.mount('/content/drive')

#Cargamos los datos de un directorio local
pathdir = '/content/drive/MyDrive/Colab Notebooks/IA y AP/Proyecto/Audios/'
listdir = os.listdir(pathdir)
print(f'Cantidad de audios: {len(listdir)}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cantidad de audios: 1697


## Creacion del vector de Etiquetas
---

In [4]:
#funcion para generar las etiquetas
def etiquetas_audios(list_audios):
  """
  Esta función tiene como input la lista
  de los nombres de los audios y retorna
  los nombres sin su numero y .mp4
  """
  names = []
  for audio in list_audios:
    split_name = re.split(r'\d', audio, maxsplit=2)
    names.append(split_name[0])
  return names

In [5]:
nombre_cetace = etiquetas_audios(listdir)

## Creación de matriz de audios
---

In [6]:
#convertimos los audios a una matriz shape(109620,2)
def audio2matriz(cetaceo, filas):
  """
  Esta función tiene como input el audio y 
  la cantidad de filas deseadas para retornar
  una matriz de shape(filas, 2) donde las
  dos columansa son amplitud y timepo
  """
  y_1, sr_1 = librosa.load(pathdir+cetaceo)
  duration = len(y_1)/sr_1
  sr = filas/duration 
  y_1 = librosa.resample(y_1, sr_1, sr)
  #algunas veces los decimales no alcanzan y sale con una fila mas
  if len(y_1) == filas+1:
    y_1 = y_1[:filas]
  #convierte amplitudes a tiempos
  amplitu2time = lambda y_1, sr : list(map(lambda x : x/sr,range(len(y_1))))
  time = amplitu2time(y_1, sr)
  #concatena amplitudes y tiempos
  Ampli0time = lambda A, t: np.concatenate((np.array(A).reshape(-1, 1), np.array(t).reshape(-1, 1)), axis= 1)
  Matriz = Ampli0time(y_1, time)
  
  return Matriz



## Creación del Tensor de Audios
---
Se busca crear matrices para cada uno de los audios con el mismo tamaño para poder apilarlos para formar un tensor de dimensiones: (1602, 109620, 2). dos columnas que representan los puntos de amplitudes y el tiempos de 109620 filas de sampleos para 1602 audios.

In [7]:
def tupla_tensors(desde, hasta):
  """
  Esta función tiene como input el idx incial 
  y el idx final de la lista de audios listdir
  y retorna una tupla de dos tensores de tensorflow
  del vector de nombres y la matriz de audios 
  """
  M = audio2matriz(listdir[desde], 109620)
  M = M[np.newaxis, :, :]
  nombre_cetace = etiquetas_audios(listdir[desde:hasta])
  for audio in range(desde+1, hasta):
    M_2 = audio2matriz(listdir[audio], 109620)
    M_2 = M_2[np.newaxis, :, :]
    M = np.concatenate((M, M_2), axis=0)
  yield (tf.constant(nombre_cetace), tf.constant(M))
        

In [8]:
Tensor_data = tupla_tensors(0,3)
print(next(Tensor_data))

(<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'Humpback Whale', b'Humpback Whale', b'Humpback Whale'],
      dtype=object)>, <tf.Tensor: shape=(3, 109620, 2), dtype=float64, numpy=
array([[[ 4.49563786e-02,  0.00000000e+00],
        [ 6.17534816e-02,  6.89634487e-05],
        [ 5.63177690e-02,  1.37926897e-04],
        ...,
        [ 1.72668621e-02,  7.55956635e+00],
        [ 2.19469201e-02,  7.55963532e+00],
        [ 7.48272240e-03,  7.55970428e+00]],

       [[ 3.68937780e-03,  0.00000000e+00],
        [ 1.20888213e-02,  1.56908156e-04],
        [ 1.37021476e-02,  3.13816313e-04],
        ...,
        [-2.36057267e-02,  1.71998014e+01],
        [-2.23530717e-02,  1.71999583e+01],
        [ 0.00000000e+00,  1.72001152e+01]],

       [[ 2.89834309e-02,  0.00000000e+00],
        [ 4.94505689e-02,  1.40939986e-04],
        [ 4.60001454e-02,  2.81879972e-04],
        ...,
        [ 8.60219747e-02,  1.54494184e+01],
        [ 9.03071985e-02,  1.54495594e+01],
        [ 7.36405477

Creamos nuestro de tensorflow dataset

In [9]:
Tensor_data = tupla_tensors(0,5) #toma los primero 5 audios
dataset0cetaceo = tf.data.Dataset.from_tensor_slices(next(Tensor_data))


In [10]:
print(f'tupla: \n{dataset0cetaceo.element_spec}') #tupla de info (etiquetas, matriz datos)

tupla: 
(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(109620, 2), dtype=tf.float64, name=None))


In [11]:
#para ver los elementos de etiquetas + matriz de datos
print(list(dataset0cetaceo.as_numpy_iterator())[0]) 
print(list(dataset0cetaceo.as_numpy_iterator())[1])
print(list(dataset0cetaceo.as_numpy_iterator())[2])
print(list(dataset0cetaceo.as_numpy_iterator())[3])
print(list(dataset0cetaceo.as_numpy_iterator())[4])

(b'Humpback Whale', array([[4.49563786e-02, 0.00000000e+00],
       [6.17534816e-02, 6.89634487e-05],
       [5.63177690e-02, 1.37926897e-04],
       ...,
       [1.72668621e-02, 7.55956635e+00],
       [2.19469201e-02, 7.55963532e+00],
       [7.48272240e-03, 7.55970428e+00]]))
(b'Humpback Whale', array([[ 3.68937780e-03,  0.00000000e+00],
       [ 1.20888213e-02,  1.56908156e-04],
       [ 1.37021476e-02,  3.13816313e-04],
       ...,
       [-2.36057267e-02,  1.71998014e+01],
       [-2.23530717e-02,  1.71999583e+01],
       [ 0.00000000e+00,  1.72001152e+01]]))
(b'Humpback Whale', array([[2.89834309e-02, 0.00000000e+00],
       [4.94505689e-02, 1.40939986e-04],
       [4.60001454e-02, 2.81879972e-04],
       ...,
       [8.60219747e-02, 1.54494184e+01],
       [9.03071985e-02, 1.54495594e+01],
       [7.36405477e-02, 1.54497003e+01]]))
(b'Humpback Whale', array([[ 1.20717008e-03,  0.00000000e+00],
       [ 2.84732017e-03,  1.52181045e-05],
       [ 5.15410537e-03,  3.04362090e-05],